<a href="https://colab.research.google.com/github/monirpust/cyber-attak-detection-using-ml-dl/blob/main/ensemble_methodology_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Anomaly detection using ML, DL, and Ensemble Methods for Gas Pipeline dataset 

# Importing necessary libraries

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split



# Importing Dataset

In [ ]:
names = ['address','function', 'length', 'setpoint', 'gain', 'reset rate', 'deadband', 'cycle time', 'rate', 'system mode', 'control scheme', 'pump', 'solenoid','pressure measurement','crc rate','command response','time','binary result','categorized result','specific result' ]
df = pd.read_csv('/content/drive/MyDrive/dataset/Stage1Test.csv',names=names,na_values='?',index_col=None);
#df = data.dropna();

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.describe()
df.head()

,address,function,length,setpoint,gain,reset rate,deadband,cycle time,rate,system mode,control scheme,pump,solenoid,pressure measurement,crc rate,command response,time,binary result,categorized result,specific result
0,address,function,length,setpoint,gain,'reset rate',deadband,'cycle time',rate,'system mode','control scheme',pump,solenoid,'pressure measurement','crc rate','command response',time,'binary result',NaN,NaN
1,4,3,16,10.535523,112.986771,0.305917,0.48863,1.050153,0.012535,0.491108,0.657535,0.318222,0.461201,727227219777317100000000000000000000,12869,1,1418682163.170388,0,NaN,NaN
2,4,3,16,10.535523,112.986771,0.305917,0.48863,1.050153,0.012535,0.491108,0.657535,0.318222,0.461201,727227219777317100000000000000000000,12869,1,1418810954.338649,0,NaN,NaN
3,4,16,16,10.535523,112.986771,0.305917,0.48863,1.050153,0.012535,0.491108,0.657535,0.318222,0.461201,727227219777317100000000000000000000,17718,0,1418828890.946774,0,NaN,NaN
4,4,3,46,10.535523,112.986771,0.305917,0.48863,1.050153,0.012535,0.491108,0.657535,0.318222,0.461201,0.574713,13360,0,1418787429.349601,0,NaN,NaN


# Data Manipulation

In [ ]:
df1=df.sort_values(by=['command response'])
cr_col=np.array(df1['command response'])
indices=np.where(cr_col==1)
splitter=indices[0][0]

df_res,df_com=df1.head(splitter),df1.tail(len(df1)-splitter)
#print (df_res.head(6))
#print (splitter)
df_temp=df_com[['address','function', 'length', 'setpoint', 'gain', 'reset rate', 'deadband', 'cycle time', 'rate', 'system mode', 'control scheme', 'pump', 'solenoid','crc rate','binary result','categorized result']].dropna()

df_temp1=df_temp[['address','function', 'length', 'setpoint', 'gain', 'reset rate', 'deadband', 'cycle time', 'rate', 'system mode', 'control scheme', 'pump', 'solenoid','crc rate']]
df_temp2=df_temp[['binary result','categorized result']]
input_com=np.array(df_temp1)
output_com=np.array(df_temp2)


df_temp=df_res[['address','function', 'length', 'pressure measurement','crc rate','binary result','categorized result']].dropna(axis=0, how='any')
#df_temp=df_temp.dropna(axis=0, how='any')

df_temp1=df_temp[['address','function', 'length', 'pressure measurement','crc rate']]
df_temp2=df_temp[['binary result','categorized result']]
input_res=np.array(df_temp1)
output_res=np.array(df_temp2)

#print (df_temp)
print (input_com.shape, output_com.shape, input_res.shape, output_res.shape)
#df_temp.isnull()

#input1=np.array(input)
#output=df[['binary result','categorized result']]
#print (input)

#Dividing Dataset

In [ ]:
## Dividing the dataset for command type packets

ntr1 = int(input_com.shape[0]/2)
nts1 = input_com.shape[0]-ntr1
Xtr1 = input_com[:ntr1,:]
ytr1 = output_com[:ntr1,1]
Xts1 = input_com[:nts1,:]
yts1 = output_com[:nts1,1]
print (Xtr1.shape,ytr1.shape)

#Trying logistic regression
logreg = linear_model.LogisticRegression()
logreg.fit(Xtr1,ytr1)
yhat = logreg.predict(Xts1)
acc = np.mean(yhat == yts1)
#print (input_com[0]==input_com[32051])
print("Accuracy on training data =", acc)
Cs=np.logspace(5,10,5)
acc_list=[]
for C_element in Cs:
    #print (C_element)
    logreg = linear_model.LogisticRegression(C=C_element)
    logreg.fit(Xtr1,ytr1)
    yhat = logreg.predict(Xts1)
    acc_list.append(np.mean(yhat == yts1))
print (acc_list)
plt.plot(Cs,acc_list)
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
#split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, data['binary result'], test_size = 0.2, random_state=0)

# Fit the Isolation Forest model on the training data
model = IsolationForest(random_state=0)
model.fit(X_train)

# Use the model to predict on the test data
y_pred = model.predict(X_test)

# Check the accuracy of the model on the test data
print("Accuracy:", np.mean(y_pred == y_test))

names = ['address','function', 'length', 'setpoint', 'gain', 'reset rate', 'deadband', 'cycle time', 'rate', 'system mode', 'control scheme', 'pump', 'solenoid','pressure measurement','crc rate','command response','time','binary result','categorized result','specific result' ]
df = pd.read_csv('C:/Users/userlocal/Desktop/PhD/Courses/Machine_Learning/IntrusionDetection/Datasets/gas_pipeline.csv',names=names,na_values='?',index_col=None)
#df = df.dropna()
#data=np.array(df)


print (df.head(6))


ValueError: ignored